# TAREA 2:

1.- Hacer un modelo regresión logistica con woes con la tabla que generaron en la tarea 1.

2.- Hacer un árbol de clasificación con la misma tabla que el problema 1. No olviden que tienen que hacer el ajuste de hiperparámetros.

3.- Seleccionar con validación cruzada cuál es el mejor modelo de los dos problemas anteriores.

4.- Obtener los puntos óptimos de decisión para el modelo seleccionado.

En todos los casos obtener: Gini, KS, ROC en entrenamiento y validación.
Fecha de entrega: Viernes 7 de enero.


## 1.- Modelo de Regresión Logística con Woes

In [1]:
# Importamos las herramientas que utilizaremos

import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from pylab import*
import seaborn as sns

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from pylab import*
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import sqldf
import scipy.stats
from scipy.special import gamma
from scipy.stats import beta
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

In [2]:
# Mandamos a llamar a la base que analizamos en la Tarea 1 esta es la que decidimos que tiene los 
# criterios necesarios y a la que se le hará la regresión. 

ruta = "C:/Users/cresp/Documents/GitHub/Seminario_9273/Tarea 2/"
base = "Base_Final.csv"
bd = pd.read_csv(ruta+base)

In [3]:
bd.head()

,Unnamed: 0,index,num_caso,edad_credit,retrasos_max,comp_interno1
0,0,0,1,62.0,1.0,3039.513678
1,1,1,7,29.0,6.0,1547.112462
2,2,2,8,63.0,1.0,1215.805471
3,3,3,9,15.0,0.0,0.000000
4,4,4,10,16.0,1.0,0.000000


In [4]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48932 entries, 0 to 48931
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     48932 non-null  int64  
 1   index          48932 non-null  int64  
 2   num_caso       48932 non-null  int64  
 3   edad_credit    48932 non-null  float64
 4   retrasos_max   48932 non-null  float64
 5   comp_interno1  48932 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 2.2 MB


In [5]:
#Hacemos validación y test
# X = bd.iloc[:,0:4].values
# y = bd.iloc[:, 5].values

In [6]:
# y

In [25]:
pip install optbinning

  Using cached optbinning-0.13.0-py3-none-any.whl (218 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached tdigest-0.5.2.2-py3-none-any.whl (9.4 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached ortools-9.2.9972-cp38-cp38-win_amd64.whl (53.3 MB)
  Using cached ropwr-0.2.0-py3-none-any.whl (14 kB)
  Using cached protobuf-3.19.1-cp38-cp38-win_amd64.whl (895 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached cvxpy-1.1.17-cp38-cp38-win_amd64.whl (852 kB)
  Using cached osqp-0.6.2.post0-cp38-cp38-win_amd64.whl (162 kB)
  Using cached ecos-2.0.8-cp38-cp38-win_amd64.whl (73 kB)
  Using cached scs-3.0.0.tar.gz (27.5 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\cresp\anaconda3\python.exe' 'C:\Users\cresp\anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\cresp\AppData\Local\Temp\tmpj5h57oq3'
       cwd: C:\Users\cresp\AppData\Local\Temp\pip-install-_09mronj\scs_116d278f5c21458aaadacaef26a2ad28
  Complete output (82 lines):
  Namespace(blas64=False, extraverbose=False, float32=False, gpu=False, gpu_atrans=True, int32=False, scs=False)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\scs
  copying src\__init__.py -> build\lib.win-amd64-3.8\scs
  running build_ext
  blas_mkl_info:
    NOT AVAILABLE
  blis_info:
    NOT AVAILABLE
  openblas_info:
      library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas']
      libraries = ['openblas']
      language = f77
      define_macros = [('HAVE_CBLAS', None)]
  blas_opt_info:
      library_dir


  Using cached pyudorandom-1.0.0-py3-none-any.whl
  Running setup.py clean for accumulation-tree
Failed to build scs accumulation-tree


In [7]:
pip install --upgrade https://github.com/jkbr/httpie/tarball/master https://github.com/guillermo-navas-palencia/optbinning.git

Note: you may need to restart the kernel to use updated packages.


  ERROR: Cannot unpack file C:\Users\cresp\AppData\Local\Temp\pip-unpack-0ho98gh0\optbinning.git (downloaded from C:\Users\cresp\AppData\Local\Temp\pip-req-build-2qo_bwbs, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of C:\Users\cresp\AppData\Local\Temp\pip-req-build-2qo_bwbs


  Using cached https://github.com/jkbr/httpie/tarball/master


In [8]:
from optbinning import OptimalBinning

ModuleNotFoundError: No module named 'optbinning'

In [27]:
train, test = train_test_split(bd, test_size = 0.30, random_state = 100) 

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34252 entries, 8006 to 38408
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     34252 non-null  int64  
 1   index          34252 non-null  int64  
 2   num_caso       34252 non-null  int64  
 3   edad_credit    34252 non-null  float64
 4   retrasos_max   34252 non-null  float64
 5   comp_interno1  34252 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 1.8 MB


In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14680 entries, 47579 to 30452
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     14680 non-null  int64  
 1   index          14680 non-null  int64  
 2   num_caso       14680 non-null  int64  
 3   edad_credit    14680 non-null  float64
 4   retrasos_max   14680 non-null  float64
 5   comp_interno1  14680 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 802.8 KB


In [30]:
y = train.comp_interno1

In [31]:
columnas = list(bd.columns.values)
print(columnas)

['Unnamed: 0', 'index', 'num_caso', 'edad_credit', 'retrasos_max', 'comp_interno1']


In [32]:
optb_num_caso = OptimalBinning(name="num_caso", dtype="numerical", solver="cp")
optb_edad_credit = OptimalBinning(name="edad_credit", dtype="categorical", solver="cp",)
optb_retrasos_max = OptimalBinning(name="retrasos_max", dtype="numerical", solver="cp")
optb_comp_interno1 = OptimalBinning(name="comp_interno1", dtype="numerical", solver="cp")

NameError: name 'OptimalBinning' is not defined

In [33]:
optb_num_caso.fit(train["num_caso"].values, y)
optb_edad_credit.fit(train["edad_credit"].values, y)
optb_retrasos_max.fit(train["retrasos_max"].values, y)
optb_comp_interno1.fit(train["comp_interno1"].values, y)


NameError: name 'optb_num_caso' is not defined